In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()

In [5]:
print(f'length of the input text for pretraining: {len(shakespeare)}.')
print(shakespeare[:1000])

length of the input text for pretraining: 1115394.
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak thi

In [6]:
# List all unique characters that occurs in the input text
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)
print('ALL chars:' + ''.join(chars))
print()
print(f'TOTAL CHARS NUM: {vocab_size}')

ALL chars:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

TOTAL CHARS NUM: 65


In [7]:
# mapping from characters to integers for encoding
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # Take a string and output a list of integers
decode = lambda i : ''.join([itos[c] for c in i]) # Take a list of integers and output a list of string

test_string = "hii, there"
print(encode(test_string))
print(decode(encode(test_string)))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43]
hii, there


In [8]:
# encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(shakespeare), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x000001F4CD694400>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61, 

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'input: {context}, target: {target}')

input: tensor([18]), target: 47
input: tensor([18, 47]), target: 56
input: tensor([18, 47, 56]), target: 57
input: tensor([18, 47, 56, 57]), target: 58
input: tensor([18, 47, 56, 57, 58]), target: 1
input: tensor([18, 47, 56, 57, 58,  1]), target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will be process in parallel?
block_size = 8 # What is the maximum context length for predictions?

In [13]:
ix = torch.randint(len(data) - block_size, (batch_size,))
ix

tensor([1078327,  453969,   41646,  671252])

In [14]:
torch.stack([data[i:i+block_size] for i in ix])

tensor([[59, 52, 49, 47, 52, 42,  1, 40],
        [53, 54, 43, 44, 59, 50,  1, 50],
        [27, 24, 33, 25, 26, 21, 13, 10],
        [47, 41, 43,  1, 53, 60, 43, 56]])

In [15]:
torch.stack([data[i+1:i+block_size+1] for i in ix])

tensor([[52, 49, 47, 52, 42,  1, 40, 56],
        [54, 43, 44, 59, 50,  1, 50, 39],
        [24, 33, 25, 26, 21, 13, 10,  0],
        [41, 43,  1, 53, 60, 43, 56, 58]])

In [16]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])


In [94]:
for b in range(batch_size): # batch dimension
    print(f'batch {b+1}/{batch_size}')
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
    print()

batch 1/4
when input is [57] the target: 43
when input is [57, 43] the target: 60
when input is [57, 43, 60] the target: 43
when input is [57, 43, 60, 43] the target: 52
when input is [57, 43, 60, 43, 52] the target: 1
when input is [57, 43, 60, 43, 52, 1] the target: 63
when input is [57, 43, 60, 43, 52, 1, 63] the target: 43
when input is [57, 43, 60, 43, 52, 1, 63, 43] the target: 39

batch 2/4
when input is [60] the target: 43
when input is [60, 43] the target: 42
when input is [60, 43, 42] the target: 8
when input is [60, 43, 42, 8] the target: 0
when input is [60, 43, 42, 8, 0] the target: 25
when input is [60, 43, 42, 8, 0, 25] the target: 63
when input is [60, 43, 42, 8, 0, 25, 63] the target: 1
when input is [60, 43, 42, 8, 0, 25, 63, 1] the target: 45

batch 3/4
when input is [56] the target: 42
when input is [56, 42] the target: 5
when input is [56, 42, 5] the target: 57
when input is [56, 42, 5, 57] the target: 1
when input is [56, 42, 5, 57, 1] the target: 57
when input is

In [17]:
# A basic neural network for bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        # the embedding table is in the shape of (vocab_size, vocab_size)
        #   a b c ... z
        # a 
        # b
        # .
        # z
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and target are in the size of (B, T)
        logits = self.token_embedding_table(idx) # (B, T, vocab_size)
        
        if targets is None: # for generation without providing target
            loss = None
        else:
            # Reshape the logits tensor to meet definition of the cross_entropy function in Pytorch
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Concatenate B, T
            targets = targets.view(B*T)    # Same reshaping to the target tensor
            loss = F.cross_entropy(logits, targets) # calculate the loss
        
        return logits, loss
    
    def generation(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx
    
n = BigramLanguageModel(vocab_size)
out, loss = n(xb, yb)
print(out.shape)
print(loss)
print()

logits = n(torch.zeros((1, 1), dtype=torch.long))[0] # (B, T) = (1, 1)
print(logits)  # logits size (1, 65, 1)
print(logits[:, -1, :]) # focus only on the final step (1, 65)
probs = F.softmax(logits, dim=-1)
print(probs)
next_idx = torch.multinomial(probs[0], num_samples=1)[0].tolist()
print(next_idx) # (1, 1)
print(decode(next_idx))

torch.Size([32, 65])
tensor(4.8549, grad_fn=<NllLossBackward0>)

tensor([[[-0.5024, -0.2026, -1.5671, -1.0980,  0.2599, -1.0698,  1.6305,
           0.7277, -0.3676, -0.7483, -0.1006,  0.7307, -0.2251,  1.3296,
          -1.2435, -0.1680, -0.0476, -1.0996, -1.7524, -1.0971, -1.1081,
          -1.8002, -0.4713,  0.0084,  0.1662,  1.2055,  0.1883, -2.1600,
          -0.1585, -0.6300, -0.2221,  0.6924,  1.1490,  0.1812, -0.0920,
           1.5828,  0.1526,  0.3843,  1.3091,  0.4645, -0.7296, -1.5580,
          -0.3950,  0.6112, -1.7097, -0.0826, -0.0495, -1.4480,  0.4109,
          -0.2422,  0.4208,  0.2776, -0.4440, -0.3797,  0.4039, -1.1757,
          -0.2749,  1.4738,  0.0688,  1.3327, -0.9523,  0.2588,  0.1108,
           0.3925,  0.8353]]], grad_fn=<EmbeddingBackward0>)
tensor([[-0.5024, -0.2026, -1.5671, -1.0980,  0.2599, -1.0698,  1.6305,  0.7277,
         -0.3676, -0.7483, -0.1006,  0.7307, -0.2251,  1.3296, -1.2435, -0.1680,
         -0.0476, -1.0996, -1.7524, -1.0971, -1.1081, -

In [18]:
print(decode(n.generation(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


MXoN?jLDJIcLVR'JIHDTHdhsV
v
wxh,nhUYZzAEOZHpgo3q3ZYZes$zuGw,;eMk QqACRfCLgxiW3.O!zDLgA YsTb!dHb!;pKeeJyYQQOrFA&$zw!:EwWSaxWEbjPQWeLc.vyN$zKr$P-Jc:'Sa3q'BK-gSM?rKroeZfrkP&:ql-l-BvtD$LNFr!DsUKaKWspvBp:ZQsWZRlkVjxt qgoKX-PjSeL $.liRaVjPspVR!mpJO!O!u'kNfEWFMPy'hjCWhB.w.VBYlDNE.tIDeDlx
?$Ox?rT.fHJxGoF&t usA?IvJFzsVDRgaNwSmEQaftEaveBDeTdvv&JpHk dq';yhPVjwy.R!ZMys-Ngmb,tBOEaGLoPeF;?xtr!LJoCMwX'FUYOxxhvHAYuSKEn.XMhn:Dh3MGFrSnIr!SXJ?BeMM!JciMPHMP!HbyfNqAbqS&LtsxhkDl3
W'yfdJoCuOqACeFpLkyJBQOVb:do3h$zAeV
:


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(n.parameters(), lr=1e-3)

In [20]:
from tqdm import tqdm

batch_size = 32
epochs = 10000
losses = []
for e in tqdm(range(epochs)):
    xb, yb = get_batch('train')
    
    logits, loss = n(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

100%|██████████| 10000/10000 [00:11<00:00, 859.44it/s]


In [21]:
print(loss.item())

2.4409990310668945


In [23]:
print(decode(n.generation(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


'shons ppplirn!
meftou ow uring, avewist th;
TENTEMETCingmenco, An he waro whiougou he s imaror?
Bu ne-ingof acat nd l,
Fothind at wrt:
HMovint gros!
AMQUThes med thestw cos wand herf s hafold madlous jouney biPoeronngabsestouMOLAgm IUNONDohery ththe tonty th, fourf thatys ng dHepp qur ace Einowhemy azer:
I,
Ishit tingthat ha tteredef seariomams.
Makine,

Than ts hientr?
3iveance re y l, urest codauseabertierr,
her tr fed
th h'es?
S:
Awo!
I'l,
Aty kise ug, ben telo,
ANG cousk, thoa hroro s lly n
